In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from ipywidgets import widgets
from IPython.display import display, clear_output
import os

# Set directories
DATASET_DIR = './dataset/'
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Define precautions dictionary
precautions = {
    "rice leaf roller": "Use light traps to monitor population and apply selective insecticides if necessary.",
    "rice leaf caterpillar": "Spray biological pesticides like Bacillus thuringiensis and maintain field hygiene.",
    "paddy stem maggot": "Drain excess water from fields and apply recommended insecticides.",
    "asiatic rice borer": "Adopt pheromone traps and use insecticides at the early larval stage.",
    "yellow rice borer": "Implement biological control using egg parasitoids and avoid over-fertilization.",
    "rice gall midge": "Plant resistant varieties and apply insecticides before egg-laying starts.",
    "brown plant hopper": "Maintain a balanced use of nitrogen fertilizers and use resistant varieties.",
    "rice stem fly": "Adjust planting dates to avoid peak pest activity and destroy crop residues after harvest.",
    "rice water weevil": "Flood fields after seeding and use seed treatments with insecticides.",
    "rice leaf hopper": "Ensure proper spacing during planting and use resistant rice varieties.",
    "rice shell pest": "Hand-pick visible pests and spray insecticides as needed.",
    "thrips": "Keep fields weed-free and use sticky traps to monitor and reduce their population.",
    "healthy": "No issues detected. Maintain regular crop monitoring."
}

# Load the trained model
try:
    model = load_model('pest_detection_model.h5')
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)

# Function to predict image and return precaution

def predict_image_with_precautions(image_path):
    image = Image.open(image_path).resize(IMG_SIZE)
    image_array = np.expand_dims(np.array(image) / 255.0, axis=0)
    prediction = model.predict(image_array)
    predicted_class = list(precautions.keys())[np.argmax(prediction)]
    confidence = np.max(prediction)
    precaution = precautions.get(predicted_class, "No precaution available for this pest.")
    return predicted_class, confidence, precaution

# File upload widget
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
output_widget = widgets.Output()

def upload_and_predict(change):
    with output_widget:
        clear_output()
        for file_name in upload_widget.value:
            file_path = f"temp_{file_name}"
            with open(file_path, 'wb') as f:
                f.write(upload_widget.value[file_name]['content'])

            predicted_class, confidence, precaution = predict_image_with_precautions(file_path)
            display(Image.open(file_path))
            print(f"Predicted Pest: {predicted_class} (Confidence: {confidence:.2f})")
            print(f"Precaution: {precaution}")
            os.remove(file_path)

upload_widget.observe(upload_and_predict, names='value')
display(widgets.VBox([upload_widget, output_widget]))

# Webcam capture and predict
def capture_and_predict():
    cap = cv2.VideoCapture(0)
    print("Press 'c' to capture and 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        cv2.imshow("Webcam", frame)
        key = cv2.waitKey(1)
        if key == ord('c'):
            captured_image_path = "captured_image.jpg"
            cv2.imwrite(captured_image_path, frame)
            print(f"Image saved as {captured_image_path}")

            predicted_class, confidence, precaution = predict_image_with_precautions(captured_image_path)
            print(f"Predicted Pest: {predicted_class} (Confidence: {confidence:.2f})")
            print(f"Precaution: {precaution}")
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Webcam capture button
webcam_button = widgets.Button(description="Capture from Webcam")

def start_webcam(b):
    capture_and_predict()

webcam_button.on_click(start_webcam)
display(webcam_button)
